# Description

Now `cdist_parts` has been optimized with previous profiling tests.

Here we profile function `_get_parts`.

Here I try a completely new `rank` function.
I'm also trying a slightly different `run_quantile_clustering`, given the changes to `rank`.
I'm also parallelizing `_get_parts` inside `_cm`.

So **be carreful** with this version.

# Remove pycache dir

In [1]:
!echo ${CODE_DIR}

/opt/code


In [2]:
!find ${CODE_DIR} -regex '^.*\(__pycache__\)$' -print

/opt/code/libs/clustermatch/__pycache__


In [3]:
!find ${CODE_DIR} -regex '^.*\(__pycache__\)$' -exec rm -rf {} \;

find: ‘/opt/code/libs/clustermatch/__pycache__’: No such file or directory


In [4]:
!find ${CODE_DIR} -regex '^.*\(__pycache__\)$' -print

# Modules

In [5]:
import numpy as np

from clustermatch.coef import _cm

# Settings

In [6]:
N_REPS = 10

In [7]:
np.random.seed(0)

# Setup

In [8]:
# let numba compile all the code before profiling
_cm(np.random.rand(10), np.random.rand(10))

(array([0.15625]),
 array([[0, 1]], dtype=uint64),
 array([[[0, 1, 1, 0, 0, 1, 0, 1, 1, 0],
         [1, 2, 1, 1, 0, 2, 0, 2, 2, 0]],
 
        [[1, 0, 0, 1, 0, 0, 0, 1, 1, 1],
         [2, 1, 1, 2, 0, 0, 0, 2, 1, 2]]], dtype=int16))

# Run with `n_samples` small

In [9]:
N_SAMPLES = 100

In [10]:
x = np.random.rand(N_SAMPLES)
y = np.random.rand(N_SAMPLES)

In [11]:
def func():
    for i in range(N_REPS):
        # py_func accesses the original python function, not the numba-optimized one
        # this is needed to be able to profile the function
        _cm(x, y)

In [12]:
%%timeit -n1 -r4 func()
func()

22 ms ± 417 µs per loop (mean ± std. dev. of 4 runs, 1 loop each)


In [13]:
%%prun -s cumulative -l 20 -T 07-n_samples_small.txt
func()

 
*** Profile printout saved to text file '07-n_samples_small.txt'. 


         14 function calls in 0.034 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.034    0.034 {built-in method builtins.exec}
        1    0.000    0.000    0.034    0.034 <string>:1(<module>)
        1    0.000    0.000    0.034    0.034 691993785.py:1(func)
       10    0.033    0.003    0.033    0.003 coef.py:254(_cm)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}

In this case (small number of samples), `cdist_parts` is still the most consuming function, followed by `rank` (`tottime`).

# Run with `n_samples` large

In [14]:
N_SAMPLES = 100000

In [15]:
x = np.random.rand(N_SAMPLES)
y = np.random.rand(N_SAMPLES)

In [16]:
def func():
    for i in range(N_REPS):
        # py_func accesses the original python function, not the numba-optimized one
        # this is needed to be able to profile the function
        _cm(x, y)

In [17]:
%%timeit -n1 -r4 func()
func()

5.44 s ± 115 ms per loop (mean ± std. dev. of 4 runs, 1 loop each)


In [18]:
%%prun -s cumulative -l 20 -T 07-n_samples_large.txt
func()

 
*** Profile printout saved to text file '07-n_samples_large.txt'. 


         14 function calls in 5.605 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    5.605    5.605 {built-in method builtins.exec}
        1    0.000    0.000    5.605    5.605 <string>:1(<module>)
        1    0.000    0.000    5.605    5.605 691993785.py:1(func)
       10    5.605    0.560    5.605    0.560 coef.py:254(_cm)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}

**Large improvement** using a new `rank` function and parallelizing the call of `_get_parts` from `_cm`.